In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
df = pd.read_csv('data/final_combined_dataset.csv')

In [3]:
df['week'] = pd.to_numeric(df['week'], errors='coerce')
df['date'] = pd.to_datetime(df['date'])

In [4]:
# Add date_ordinal
df['date_ordinal'] = df['date'].apply(lambda x: x.toordinal())

# Extract year and month
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Add cyclic month representation
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# Add cyclic week representation
df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52234 entries, 0 to 52233
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date                                  52234 non-null  datetime64[ns]
 1   week                                  52234 non-null  int64         
 2   cases                                 52234 non-null  int64         
 3   population                            52234 non-null  int64         
 4   tempe_min                             52234 non-null  float64       
 5   humidity_max                          52234 non-null  float64       
 6   humidity_avg                          52234 non-null  float64       
 7   humidity_min                          52234 non-null  float64       
 8   temp_avg                              52234 non-null  float64       
 9   temp_max                              52234 non-null  float64       
 10

In [6]:
geocode_list = [3303807]
df = df[df['geocode'].isin(geocode_list)]
df = df.drop(columns=['geocode'])

In [7]:
df['cases_lag1'] = df['cases'].shift(1)
df['cases_lag2'] = df['cases'].shift(2)
df['cases_lag3'] = df['cases'].shift(3)
df['cases_lag4'] = df['cases'].shift(4)

# lag nearby_cases_weighted 4 weeks
df['nearby_cases_weighted_lag1'] = df['nearby_cases_weighted'].shift(1)
df['nearby_cases_weighted_lag2'] = df['nearby_cases_weighted'].shift(2)
df['nearby_cases_weighted_lag3'] = df['nearby_cases_weighted'].shift(3)
df['nearby_cases_weighted_lag4'] = df['nearby_cases_weighted'].shift(4)

df

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,...,week_sin,week_cos,cases_lag1,cases_lag2,cases_lag3,cases_lag4,nearby_cases_weighted_lag1,nearby_cases_weighted_lag2,nearby_cases_weighted_lag3,nearby_cases_weighted_lag4
0,2012-01-01,201201,32,207044,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,...,1.000000,7.185429e-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-08,201202,40,207044,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,...,0.992709,-1.205367e-01,32.0,NaN,NaN,NaN,6.913995,NaN,NaN,NaN
2,2012-01-15,201203,19,207044,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,...,0.970942,-2.393157e-01,40.0,32.0,NaN,NaN,32.825012,6.913995,NaN,NaN
3,2012-01-22,201204,33,207044,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,...,0.935016,-3.546049e-01,19.0,40.0,32.0,NaN,14.685214,32.825012,6.913995,NaN
4,2012-01-29,201205,36,207044,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,...,0.885456,-4.647232e-01,33.0,19.0,40.0,32.0,2.809255,14.685214,32.825012,6.913995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52229,2022-11-27,202248,1,273988,21.857143,95.875069,82.840922,61.313170,25.507143,30.857143,...,0.663123,-7.485107e-01,1.0,0.0,1.0,1.0,0.000000,0.887888,2.076361,0.000000
52230,2022-12-04,202249,0,273988,21.857143,92.543531,80.151192,64.110460,24.903139,28.142857,...,0.568065,-8.229839e-01,1.0,1.0,0.0,1.0,0.000000,0.000000,0.887888,2.076361
52231,2022-12-11,202250,2,273988,21.000000,94.213747,79.480439,59.964030,24.597222,28.500000,...,0.464723,-8.854560e-01,0.0,1.0,1.0,0.0,0.938861,0.000000,0.000000,0.887888
52232,2022-12-18,202251,1,273988,21.000000,91.312065,84.229947,73.950180,23.184295,26.000000,...,0.354605,-9.350162e-01,2.0,0.0,1.0,1.0,12.281740,0.938861,0.000000,0.000000


In [8]:
df.isnull().sum()

date                                    0
week                                    0
cases                                   0
population                              0
tempe_min                               0
humidity_max                            0
humidity_avg                            0
humidity_min                            0
temp_avg                                0
temp_max                                0
city                                    0
vim                                     0
vim_monthly                             0
precipitation_avg_ordinary_kriging      0
precipitation_max_ordinary_kriging      0
precipitation_avg_regression_kriging    0
precipitation_max_regression_kriging    0
long                                    0
lat                                     0
cases_per_100k                          0
nearby_cases_weighted                   0
date_ordinal                            0
year                                    0
month                             

In [9]:
df.shape

(52234, 36)

In [10]:
df = df.dropna()

In [11]:
train_df = df[df['date'].dt.year <= 2020]
test_df = df[df['date'].dt.year >= 2021]

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

Training set shape: (42766, 36)
Test set shape: (9464, 36)


In [12]:
X_train = train_df.drop(columns=['cases_per_100k', 'cases'])
y_train = train_df[['cases', 'city']]

X_test = test_df.drop(columns=['cases_per_100k', 'cases'])
y_test = test_df[['cases', 'city']]

test_df = test_df.iloc[4:].reset_index(drop=True)

In [13]:
columns_to_not_scale = ['week_sin', 'week_cos', 'month_sin', 'month_cos', 'cases', 'cases_per_100k']
numeric_cols = [
    col for col in df.select_dtypes(include=['float64', 'int64', 'int32']).columns
    if col not in columns_to_not_scale
]

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Scale the selected numeric features
X_train[numeric_cols] = feature_scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = feature_scaler.transform(X_test[numeric_cols])

# Separate the city column from y_train and y_test (assuming 'city' is a column in y_train and y_test)
city_train = y_train['city']
city_test = y_test['city']

# Separate the target (cases) column from y_train and y_test
y_train_cases = y_train.drop('city', axis=1)  # Remove the 'city' column from y_train
y_test_cases = y_test.drop('city', axis=1)  # Remove the 'city' column from y_test

# Scale the target (cases) values
y_train = target_scaler.fit_transform(y_train_cases)
y_test = target_scaler.transform(y_test_cases)

# Recombine the city column with the scaled target values
y_train = pd.DataFrame(y_train, columns=['cases'])
y_train['city'] = city_train.reset_index(drop=True)

y_test = pd.DataFrame(y_test, columns=['cases'])
y_test['city'] = city_test.reset_index(drop=True)

In [14]:
# Feature columns you want to select
feature_cols = [
    'temp_avg', 'humidity_avg', 'cases_lag1', 'cases_lag2', 'nearby_cases_weighted_lag1', 'precipitation_max_regression_kriging', 'week_sin', 'month_sin', 'week_cos', 'month_cos']

In [15]:
# Function to create overlapping sequences
def create_sequences(data, target, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length].values)  # Select seq_length rows as input
        y.append(target.iloc[i + seq_length])  # Select the next row as the target
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 4

# Group data by city
city_groups_xtrain = X_train.groupby('city')
city_groups_xtest = X_test.groupby('city')

city_groups_ytrain = y_train.groupby('city')
city_groups_ytest = y_test.groupby('city')

# Initialize dictionaries to store the processed data for each city
X_train_citywise_seq = {}
y_train_citywise_seq = {}
X_test_citywise_seq = {}
y_test_citywise_seq = {}

# Process training data city-wise
for city, X_train_city in city_groups_xtrain:
    y_train_city = city_groups_ytrain.get_group(city)  # Get the corresponding target group
    # Reset index to ensure continuous indexing
    y_train_city = y_train_city.reset_index(drop=True)
    X_seq, y_seq = create_sequences(X_train_city[feature_cols], y_train_city['cases'], seq_length)  # Use 'cases' as target
    X_train_citywise_seq[city] = X_seq
    y_train_citywise_seq[city] = y_seq
    print(f"Processed X_train and y_train for city: {city}, X_seq shape: {X_seq.shape}, y_seq shape: {y_seq.shape}")

# Process testing data city-wise
for city, X_test_city in city_groups_xtest:
    y_test_city = city_groups_ytest.get_group(city)  # Get the corresponding target group
    # Reset index to ensure continuous indexing
    y_test_city = y_test_city.reset_index(drop=True)
    X_seq, y_seq = create_sequences(X_test_city[feature_cols], y_test_city['cases'], seq_length)  # Use 'cases' as target
    X_test_citywise_seq[city] = X_seq
    y_test_citywise_seq[city] = y_seq
    print(f"Processed X_test and y_test for city: {city}, X_seq shape: {X_seq.shape}, y_seq shape: {y_seq.shape}")


Processed X_train and y_train for city: angra dos reis, X_seq shape: (462, 4, 10), y_seq shape: (462,)
Processed X_train and y_train for city: aperibé, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: araruama, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: areal, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: armação dos búzios, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: arraial do cabo, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: barra do piraí, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: barra mansa, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: belford roxo, X_seq shape: (466, 4, 10), y_seq shape: (466,)
Processed X_train and y_train for city: bom jardim, X_seq shape: (466, 4, 10), y_seq shape: (46

In [16]:
# Define sequence length for LSTM
seq_length = 4

# Function to build and compile the model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)  # Predicting a single value (cases)
    ])
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])
    return model

# Loop through each city dataset
city_models = {}
city_histories = {}
city_predictions = {}

for city in X_train_citywise_seq.keys():
    print(f"Processing city: {city}")

    # Get city-specific data
    X_train_city = X_train_citywise_seq[city]
    y_train_city = y_train_citywise_seq[city]
    X_test_city = X_test_citywise_seq[city]
    y_test_city = y_test_citywise_seq[city]

    # Ensure input shape matches the LSTM model
    input_shape = (seq_length, X_train_city.shape[2])

    # Build the model
    model = build_lstm_model(input_shape)

    # Train the model
    history = model.fit(
        X_train_city, y_train_city,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_city, y_test_city),
        verbose=1
    )

    # Evaluate the model
    test_loss, test_mae = model.evaluate(X_test_city, y_test_city)
    print(f"City: {city}, Test Loss: {test_loss}, Test MAE: {test_mae}")

    # Save the model for this city
    model.save(f"lstm_dengue_prediction_model_{city}.h5")

    # Predict on test data
    predictions = model.predict(X_test_city)

    # Store results
    city_models[city] = model
    city_histories[city] = history
    city_predictions[city] = predictions

    print(f"Completed processing for city: {city}\n")

Processing city: angra dos reis


C:\Users\ASUS\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - loss: 6.2341e-04 - mae: 0.0191 - val_loss: 1.8910e-04 - val_mae: 0.0120
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.9871e-04 - mae: 0.0130 - val_loss: 5.0985e-05 - val_mae: 0.0061
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.7557e-04 - mae: 0.0102 - val_loss: 3.0141e-05 - val_mae: 0.0046
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.4886e-04 - mae: 0.0086 - val_loss: 2.8090e-05 - val_mae: 0.0039
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.2187e-04 - mae: 0.0075 - val_loss: 4.1051e-05 - val_mae: 0.0047
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.1471e-04 - mae: 0.0071 - val_loss: 2.3903e-05 - val_mae: 0.0035
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0466e-04 - mae: 0.0062 - val_loss: 2.1022e-05 - val_mae: 0.0031
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8.0135e-05 - mae: 0.0051 - val_loss: 1.2103e-05 - val_mae: 0.0024


City: angra dos reis, Test Loss: 2.1197915884840768e-06, Test MAE: 0.0008190259104594588
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
Completed processing for city: angra dos reis

Processing city: aperibé
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 5.1908e-05 - val_mae: 0.0055
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.9604e-04 - mae: 0.0139 - val_loss: 7.8736e-05 - val_mae: 0.0077
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9912e-04 - mae: 0.0110 - val_loss: 5.6073e-05 - val_mae: 0.0059
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.3576e-04 - mae: 0.0088 - val_loss: 3.0486e-05 - val_mae: 0.0045
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 8.7346e-05 - mae: 0.0073 - val_loss: 2.1392e-05 - val_mae: 0.0039
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 6.7006e-05 - mae: 0.0064 - val_loss: 1.0453e-05 - val_mae: 0.0026
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - lo

City: aperibé, Test Loss: 5.083652467874344e-07, Test MAE: 0.0005392868770286441
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step
Completed processing for city: aperibé

Processing city: araruama
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0016 - mae: 0.0301 - val_loss: 1.3946e-04 - val_mae: 0.0101
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.1659e-04 - mae: 0.0140 - val_loss: 9.0743e-05 - val_mae: 0.0080
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8638e-04 - mae: 0.0109 - val_loss: 2.2554e-05 - val_mae: 0.0037
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 1.1864e-04 - mae: 0.0085 - val_loss: 2.7852e-05 - val_mae: 0.0043
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7.5245e-05 - mae: 0.0066 - val_loss: 1.0534e-05 - val_mae: 0.0025
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.8022e-05 - mae: 0.0053 - val_loss: 2.6457e-05 - val_mae: 0.0043
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.2865e-05 

...

City: porto real, Test Loss: 5.125985680365375e-08, Test MAE: 0.00020869314903393388
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step
Completed processing for city: porto real

Processing city: quatis
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 4.4120e-04 - mae: 0.0168 - val_loss: 6.4266e-05 - val_mae: 0.0067
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.4902e-04 - mae: 0.0095 - val_loss: 3.3142e-05 - val_mae: 0.0050
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1098e-04 - mae: 0.0082 - val_loss: 2.6917e-05 - val_mae: 0.0045
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.5894e-05 - mae: 0.0069 - val_loss: 4.0279e-05 - val_mae: 0.0058
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7.5263e-05 - mae: 0.0070 - val_loss: 1.1861e-05 - val_mae: 0.0030
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.2627e-05 - mae: 0.0052 - val_loss: 3.5338e-06 - val_mae: 0.0016
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.

City: quatis, Test Loss: 1.7632427784519678e-07, Test MAE: 0.0003290139138698578
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step
Completed processing for city: quatis

Processing city: queimados
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 6.7999e-04 - mae: 0.0207 - val_loss: 3.3399e-05 - val_mae: 0.0050
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.3199e-04 - mae: 0.0119 - val_loss: 5.1392e-05 - val_mae: 0.0062
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.3119e-04 - mae: 0.0089 - val_loss: 6.2859e-05 - val_mae: 0.0071
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0321e-04 - mae: 0.0077 - val_loss: 2.2941e-05 - val_mae: 0.0040
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 5.4065e-05 - mae: 0.0056 - val_loss: 2.8495e-05 - val_mae: 0.0041
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.3286e-05 - mae: 0.0050 - val_loss: 1.6799e-05 - val_mae: 0.0035
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.0616

City: queimados, Test Loss: 8.673922025081993e-08, Test MAE: 0.0002927991736214608
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step
Completed processing for city: queimados

Processing city: quissamã
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0012 - mae: 0.0231 - val_loss: 1.3967e-04 - val_mae: 0.0095
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.9423e-04 - mae: 0.0109 - val_loss: 9.1892e-05 - val_mae: 0.0072
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.3174e-04 - mae: 0.0089 - val_loss: 5.4219e-05 - val_mae: 0.0057
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.1427e-05 - mae: 0.0059 - val_loss: 1.9097e-05 - val_mae: 0.0037
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.6940e-05 - mae: 0.0050 - val_loss: 6.7511e-06 - val_mae: 0.0022
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.3968e-05 - mae: 0.0041 - val_loss: 1.0901e-05 - val_mae: 0.0027
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.061

City: quissamã, Test Loss: 8.685177732559168e-08, Test MAE: 0.00020971526100765914
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step
Completed processing for city: quissamã

Processing city: resende
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0011 - mae: 0.0247 - val_loss: 1.1455e-04 - val_mae: 0.0079
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.5223e-04 - mae: 0.0115 - val_loss: 1.2008e-04 - val_mae: 0.0090
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.5582e-04 - mae: 0.0088 - val_loss: 4.2822e-05 - val_mae: 0.0054
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6739e-04 - mae: 0.0083 - val_loss: 3.4439e-05 - val_mae: 0.0046
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.3017e-04 - mae: 0.0071 - val_loss: 2.8331e-05 - val_mae: 0.0037
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0238e-04 - mae: 0.0057 - val_loss: 1.5244e-05 - val_mae: 0.0028
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.3969e-

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

# Function to plot Actual vs Predicted and calculate RMSE
def plot_actual_vs_predicted(city, y_test_city, predictions_city, target_scaler):
    # Ensure to only pass the target variable (numeric values) to the scaler
    y_test_values = y_test_city['cases'].values  # Assuming 'cases' is the target column
    predictions_original_scale = target_scaler.inverse_transform(predictions_city)
    actual_values_original_scale = target_scaler.inverse_transform(y_test_values.reshape(-1, 1))  # Reshape for scaling

    # Ensure both predictions and actual values have the same length
    min_length = min(len(actual_values_original_scale), len(predictions_original_scale))

    actual_values_original_scale = actual_values_original_scale[:min_length].flatten()
    predictions_original_scale = predictions_original_scale[:min_length].flatten()
    predictions_original_scale[predictions_original_scale < 0] = 0

    time_index = np.arange(min_length)

    # Print shapes
    print(f"City: {city} - Actual Values Shape:", actual_values_original_scale.shape)
    print(f"City: {city} - Predicted Values Shape:", predictions_original_scale.shape)

    # Plot Actual vs Predicted
    plt.figure(figsize=(12, 6))
    plt.plot(time_index, actual_values_original_scale, label="Actual Values", color='blue', linewidth=2)
    plt.plot(time_index, predictions_original_scale, label="Predicted Values", color='orange', linewidth=2, linestyle='--')
    plt.xlabel("Time (Sequential Index)")
    plt.ylabel("Cases per 100k (Original Scale)")
    plt.title(f"{city} - Actual vs Predicted Cases per 100k Over Time")
    plt.legend()
    plt.grid()
    plt.show()

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(actual_values_original_scale, predictions_original_scale))
    print(f"City: {city} - RMSE: {rmse}")

# Loop through each city's predictions and plot
for city in city_predictions.keys():
    predictions_city = city_predictions[city]
    y_test_city = city_groups_ytest.get_group(city)

    # Call the function to plot and calculate RMSE
    plot_actual_vs_predicted(city, y_test_city, predictions_city, target_scaler)